## Better performance with tf.function

In Tensorflow 2.0, eager execution is turned on by default. The use interface is intuitive and flexible (running one-off operations is much easier and faster), but this can come at the expense of performace and deployability.

To get peak performance and to make your model deployable anywhere, use `tf.function` to make graphs out of your programs. Thanks to AutoGraph, a surprising amount of Python code just works with tf.function, but there are still pitfills to be wary of.

The main takeawyas and recommendations are:
- Don't rely on Python side effects like object mutation or list appends.
- tf.function works best with TensorFlow ops, rather than NumPy ops or Python primitives.
- When in doubt, use the `for x in y` idiom

In [2]:
from __future__ import absolute_import, division, print_function

%tensorflow_version 2.x

import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
import contextlib

# Some helper code to demonstrate the kinds of errors you might encounter.
@contextlib.contextmanager
def assert_raises(error_class):
  try:
    yield
  except error_class as e:
    print('Caught expected exception \n {}: {}'.format(error_class, e))
  except Exception as e:
    print('Got unexpected exception \n {}: {}'.format(type(e), e))
  else:
    raise Exception('Expected {} to be raised but no error was raised!'.format(
        error_class))

In [4]:
# A `tf.function` is just like a core TensorFlow operation
# You can execute it eagerly; you can use it in a graph; it has gradients; and so on.

# A function is like an op
@tf.function
def add(a, b):
  return a + b
add(tf.ones([2, 2]), tf.ones([2, 2]))

<tf.Tensor: id=14, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [5]:
# Functions have gradients

v = tf.Variable(1.0)
with tf.GradientTape() as tape:
  result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: id=37, shape=(), dtype=float32, numpy=1.0>

In [6]:
# You can use functions inside functions

@tf.function
def dense_layer(x, w, b):
  return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: id=63, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

## Tracing and polymorphism
Python's dynamic typing means that you can call functions with a variety of argument types, and Python will do something different in each scenario.

One the other hand, TensorFlow graphs require static dtypes and shape dimensions. `tf.function` bridges this gap by retracing the function when necessary to generate the correct graphs. Most of the subtlety of `tf.function` usage stems from this retracing behavior.


In [8]:
# You can call a function with arguments of different types to see
# what is happening

# Functions are polymorphic

@tf.function
def double(a):
  print('Tracing with', a)
  return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant('a')))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



To control the tracing behavior, use the following techniques:

- Create a new `tf.function`. Separate `tf.function` objects are guaranteed npt to share traces.
- Use the `get_concrete_function` method to get a specific trace
- Specify `input_signiture` when calling `tf.function` to trace only once per calling graph.

In [10]:
# Traces argument information at compilation time.
print('Obtaining concrete trace')
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
print('Executing traced function')
print(double_strings(tf.constant('a')))
print(double_strings(a=tf.constant('b')))
print('Use a concrete trace with imcompatible types will throw an error')
with assert_raises(tf.errors.InvalidArgumentError):
  double_strings(tf.constant(1))

Obtaining concrete trace
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
Use a concrete trace with imcompatible types will throw an error
Caught expected exception 
 <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_94 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_94]


In [13]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
  print('Tracing with', x)
  return tf.where(x % 2 == 0, x // 2, 3 * x + 1)

print(next_collatz(tf.constant([1, 2])))
# We specified a 1-D tensor in the input signature, so this should fail.
with assert_raises(ValueError):
  next_collatz(tf.constant([[1, 2], [3, 4]]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
Caught expected exception 
 <class 'ValueError'>: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [14]:
next_collatz(tf.constant([1, 2, 3, 4]))

<tf.Tensor: id=125, shape=(4,), dtype=int32, numpy=array([ 4,  1, 10,  2], dtype=int32)>

## When to retrace?
A polymorphic `tf.function` keeps a cache of concrete functions generated by tracing. The cache keys are effectively tuples of keys generated from the function args and kwargs. The key generated for a `tf.Tensor` argument is it's shpape and type. The key generated for a Python primitive is its value. For all other Python types, the keys are based on the object `id()` so that methods are traced independently for each instance of class. In the future, TensorFlow may add more sophisticated caching for Python objects that can be safely converted to tensors.

## Python or Tensor args?
Often, Python arguments are used to control hypterparameters and graph constructions - for example, `num_layers=10` or `training=True` or `nonlinearity='relu'`. So if the Python argument changes, it make sense that you'd have to retrace the graph.

However, it's possible that a Python argument is not being used to control graph construction. In these cases, a change in the Python value can trigger needless retracing.

In [17]:
# Take, for example, this training loop, which AutoGraph will dynamically unrool.
# Despite the multiple traces, the generated graph is actually identical, so this is a bit inefficient.
def train_one_step():
  pass

@tf.function()
def train(num_steps):
  print('tracing with num_steps = {}'.format(num_steps))
  print('hi')
  for _ in tf.range(num_steps):
    train_one_step()
train(num_steps=10)
train(num_steps=20)

tracing with num_steps = 10
hi
tracing with num_steps = 20
hi


In [18]:
# The simple workaround here is to cast your arguments to Tensors
# if they do not affect the shape of the generated graph
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20)) # Retrace: 다시 만드는거.
# 그래프가 만들어지면 print문이 없어지나? oo side effect는 trace에만 일어남.

tracing with num_steps = Tensor("num_steps:0", shape=(), dtype=int32)
hi


## Side effects in tf.function
In general, Python side effects (like printing or mutating objects) only happen during tracing.
So how can you reliably trigger side effects from `tf.function`?

The general rule of thumb is to only use Python side effects to debug your traces. Otherwise, TensorFlow ops like `tf.Variable.assign`, `tf.print`, and `tf.summary` are the best way to ensure your code will be traced and executed by the TensorFlow runtime with each call. In general using a functional style will yield the best results.

In [21]:
@tf.function
def f(x):
  print('Traced with', x)
  tf.print('Executed with', x)
f(1)
f(1)
f(3)

Traced with 1
Executed with 1
Executed with 1
Traced with 3
Executed with 3


If you would like to execute Python code during each invocation of a `tf.function`, `tf.py_function` is an exit hatch. The drawback of `tf.py_function` is that it's not portable or particularily performant, nor does it work well in distributed (multi-GPU, TPU) setups. Also, since `tf.py_function` has to be wired into the graph, it casts all inputs/outputs to tensors.

In [22]:
external_list = []

def side_effect(x):
  print('Python side effect')
  external_list.append(x)

@tf.function
def f(x):
  tf.py_function(side_effect, inp=[x], Tout=[])

f(1)
f(1)
f(1)
assert len(external_list) == 3
# .numpy() call required because py_function casts 1 to tf.constant(1)
assert external_list[0].numpy() == 1

Python side effect
Python side effect
Python side effect


In [23]:
external_list = []

def side_effect(x):
  print('Python side effect')
  external_list.append(x)

@tf.function
def f(x):
  side_effect(1)

f(1)
f(1)
f(1)
assert len(external_list) == 3
# .numpy() call required because py_function casts 1 to tf.constant(1)
assert external_list[0].numpy() == 1

Python side effect


AssertionError: ignored

## Beware of Python state
Many Python features, such as generators and iterators, rely on Python runtime to keep track of state. In general, while these constructs work as expected in Eager mode, many unexpected things can happen inside a `tf.function` due to tracing behavior.

In [25]:
# To give one example, advancing iterator state is a Python side effect and
# and therefor only happens during tracing.
external_var = tf.Variable(0)
@tf.function
def buggy_consume_next(iterator):
  external_var.assign_add(next(iterator))
  tf.print("Value of external_var:", external_var)

iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value of external_var: 0
Value of external_var: 0
Value of external_var: 0


If an iterator is generated and consumed entirely within the tf.function, then it should work correctly. However, the entire iterator is probably being traced, which can lead to a giant graph. This may be what you want. But if you're training on an large in-memory dataset represented as a Python list, then this can generate a very large graph, and `tf.function` is unlikely to yield a speedup.

If you want to iterate over Python data, the safest way is to wrap it in a tf.data.Dataset and use the `for x in y` idiom. Autograph has special support for safely converting `for` loops when `y` is a tensor or `tf.data.Dataset`.

In [0]:
def measure_graph_size(f, *args):
  g = f.get_concrete_function(*args).graph
  print('{}({}) contains {} nodes in its graph'.format(
      f.__name__, ', '.join(map(str, args)), len(g.as_graph_def().node)))

In [29]:
@tf.function
def train(dataset):
  loss = tf.constant(0)
  for x, y in dataset:
    loss += tf.abs(y - x) # Some dummy computation
  return loss

small_data = [(1, 1)] * 2
big_data = [(1, 1)] * 10
measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

train([(1, 1), (1, 1)]) contains 8 nodes in its graph
train([(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]) contains 32 nodes in its graph


In [32]:
small_dataset = tf.data.Dataset.from_generator(lambda: small_data, (tf.int32, tf.int32))
big_dataset = tf.data.Dataset.from_generator(lambda: big_data, (tf.int32, tf.int32))

measure_graph_size(train, small_dataset)
measure_graph_size(train, big_dataset)

train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 5 nodes in its graph
train(<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>) contains 5 nodes in its graph


In [31]:
print(small_dataset)

<DatasetV1Adapter shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>


When wrapping Python/Numpy data in a Dataset, be mindful of `tf.data.Dataset.from_generator` versus `tf.data.Dataset.from_tensors`. The former will keep the data in Python and fetch it via `tf.py_function` which have performance implications, whereas the latter will bundle a copy of the data as one large `tf.constant()` node in graph, which can have memory implications.

Reading data from files via TFRecordDataset/CsvDataset

When wrapping Python/Numpy data in a Dataset, be mindful of tf.data.Dataset.from_generator versus tf.data.Dataset.from_tensors. The former will keep the data in Python and fetch it via tf.py_function which can have performance implications, whereas the latter will bundle a copy of the data as one large tf.constant() node in the graph, which can have memory implications.

Reading data from files via TFRecordDataset/CsvDataset/etc. is the most effective way to consume data, as then TensorFlow itself can manage the asynchronous loading and prefetching of data, without having to involve Python.

## Automatic Control Dependencies
A very appealing property of functions as the programming model, over a general dataflow graph, is that functions can give the runtime more information about what was the intended behavior of the code.

For example, when writing code which has multiple reads and writes to the same variables, a dataflow graph might not naturally encode the originally intended order of operations. In `tf.function`, we resolve ambiguities in execution order by referring to the execution order of statements in the original Python code. This way, ordering of stateful operations in `tf.function` replicates the semantics of Eager mode.

This means there's no noeed to manual control dependencies; `tf.function` is smart enough to add the minimal set of necessary and sufficient control dependencies for your code to run correctly.

In [33]:
# Automatic control dependencies

a = tf.Variable(1.0)
b = tf.Variable(2.0)

@tf.function
def f(x, y):
  a.assign(y * b)
  b.assign_add(x * a)
  return a + b
f(1.0, 2.0)

<tf.Tensor: id=608, shape=(), dtype=float32, numpy=10.0>

## Variables
We can use the same idea of leveraging the intended execution order of the code to make variable creation and utilization very easy in tf.function. There is one very important caveat, though, which is that with variables it's possible to write code which behaves differently in eager mode and graph mode.

Specifically, this will happen when you create a new Variable with each call. Due to tracing semantics, tf.function will reuse the same variable each call, but eager mode will create a new variable with each call. To guard against this mistake, tf.function will raise an error if it detects dangerous variable creation behavior.

In [0]:
@tf.function
def f(x):
  v = tf.Variable(1.0)
  v.assign_add(x)
  return v

In [35]:
with assert_raises(ValueError):
  f(1.0)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Caught expected exception 
 <class 'ValueError'>: in converted code:

    <ipython-input-34-9ce361fb3439>:3 f  *
        v = tf.Variable(1.0)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/eager/def_function.py:413 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.



In [36]:
# Non-ambiguous code is ok though
v = tf.Variable(1.0)

@tf.function
def f(x):
  return v.assign_add(x)

print(f(1.0))
print(f(2.0))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [37]:
# You can also create variables inside a tf.function as long as we can prove
# that those variables are created only the first time the function is executed.

class C: pass
obj = C()
obj.v = None

@tf.function
def g(x):
  if obj.v is None:
    obj.v = tf.Variable(1.0)
  return obj.v.assign_add(x)

print(g(1.0))
print(g(2.0))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [38]:
# Variable initializers can depend on function arguments and on values of other
# variables. We can figure out the right initialization order using the same
# method we use to generate control dependencies.
state = []
@tf.function
def fn(x):
  if not state:
    state.append(tf.Variable(2.0 * x))
    state.append(tf.Variable(state[0] * 3.0))
  return state[0] * x * state[1]

print(fn(tf.constant(1.0)))
print(fn(tf.constant(3.0)))

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(36.0, shape=(), dtype=float32)


## Using AutoGraph
The autograph library is fully integrated with `tf.function`, and it will rewrite conditionals and loops which depend on Tensors to run dynamically in the graph.

`tf.cond` and `tf.while_loop` continue to work with `tf.function`, but code with control flow is often easier to write and understand when written in imperative style.

In [40]:
# Simple loop

@tf.function
def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    x = tf.tanh(x)
  return x

f(tf.random.uniform([5]))

[0.42983079 0.229050159 0.244930744 0.749942541 0.628233671]
[0.405179888 0.225126818 0.240147591 0.63511467 0.556834817]
[0.384372324 0.221399069 0.235635117 0.561564 0.505625248]
[0.36649853 0.217851132 0.231368706 0.509137 0.466529578]
[0.350925207 0.214468956 0.227326706 0.469272494 0.435391]
[0.337195814 0.211240053 0.223490074 0.437611312 0.409816802]
[0.324971616 0.208153114 0.219842 0.411662489 0.388317108]
[0.313995332 0.205198035 0.216367453 0.389883369 0.369908512]
[0.304067403 0.202365652 0.213053107 0.37125963 0.353911668]
[0.295030415 0.19964774 0.209886953 0.355093 0.33984]
[0.286758184 0.197036728 0.206858277 0.340884507 0.327334553]
[0.27914831 0.194525793 0.203957394 0.328266799 0.316123694]
[0.272116601 0.192108706 0.201175541 0.31696254 0.305997729]
[0.265593231 0.189779744 0.198504776 0.306757867 0.296791732]
[0.259519547 0.187533692 0.195937902 0.297484726 0.288373858]
[0.253846079 0.185365707 0.193468347 0.289009094 0.280637413]
[0.248530596 0.183271378 0.1910900

<tf.Tensor: id=865, shape=(5,), dtype=float32, numpy=
array([0.2090093 , 0.16561368, 0.17131145, 0.22730848, 0.22317973],
      dtype=float32)>

In [42]:
# If you're curious you can inspect the code autograph generates.
# It feels like reading assembly language, though.

def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    x = tf.tanh(x)
  return x

print(tf.autograph.to_code(f))

def tf__f(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('f', 'f_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as f_scope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def loop_body(x):
      ag__.converted_call(tf.print, f_scope.callopts, (x,), None, f_scope)
      x = ag__.converted_call(tf.tanh, f_scope.callopts, (x,), None, f_scope)
      return x,

    def loop_test(x):
      return ag__.converted_call(tf.reduce_sum, f_scope.callopts, (x,), None, f_scope) > 1
    x, = ag__.while_stmt(loop_test, loop_body, get_state, set_state, (x,), ('x',), ())
    do_return = True
    retval_ = f_scope.mark_return_value(x)
  do_return,
  return ag__.retval(retval_)



## AutoGraph: Conditionals
AutoGraph will convert `if` statements to the equivalent `tf.cond` calls.

This substitution is made if the condition is a Tensor. Otherwise, the conditional is executed during tracing.

In [0]:
def test_tf_cond(f, *args):
  g = f.get_concrete_function(*args).graph
  if any(node.name == 'cond' for node in g.as_graph_def().node):
    print('{}({}) use tf.cond'.format(
        f.__name__, ', '.join(map(str, args))))
  else:
    print('{}({}) executes normally.'.format(
        f.__name__, ', '.join(map(str, args))))

In [0]:
@tf.function
def hyperparam_cond(x, training=True):
  if training:
    x = tf.nn.dropout(x, rate=0.5)
  return x

In [0]:
@tf.function
def maybe_tensor_cond(x):
  if x < 0:
    x = -x
  return x

In [47]:
test_tf_cond(hyperparam_cond, tf.ones([1], dtype=tf.float32))

hyperparam_cond(tf.Tensor([1.], shape=(1,), dtype=float32)) executes normally.


In [49]:
test_tf_cond(maybe_tensor_cond, tf.constant(-1))

maybe_tensor_cond(tf.Tensor(-1, shape=(), dtype=int32)) use tf.cond


In [50]:
test_tf_cond(maybe_tensor_cond, -1)

maybe_tensor_cond(-1) executes normally.


`tf.cond` has a number of subtleties. - it works by tracing both sides of the conditional, and then choosing the appropriate branch at runtime, depending on the condition. Tracing both sides can result in unexpected execution of Python code - it requires that if one branch creates a tensor used downstream, the other branch must also create that tensor.

In [0]:
@tf.function
def f():
  x = tf.constant(0)
  if tf.constant(True):
    x = x + 1
    print('Tracing `then` branch')
  else:
    x = x - 1
    print('Tracing `else` branch')
  return x

In [52]:
f()

Tracing `then` branch
Tracing `else` branch


<tf.Tensor: id=937, shape=(), dtype=int32, numpy=1>

In [0]:
@tf.function
def f():
  if tf.constant(True):
    x = tf.ones([3, 3])
  return x

In [54]:
# Throws an error because both branches need to define `x`.
with assert_raises(ValueError):
  f()

Caught expected exception 
 <class 'ValueError'>: in converted code:

    <ipython-input-53-a6da8fb68411>:3 f  *
        if tf.constant(True):
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:893 if_stmt
        basic_symbol_names, composite_symbol_names)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:931 tf_if_stmt
        error_checking_orelse)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/util/deprecation.py:507 new_func
        return func(*args, **kwargs)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/ops/control_flow_ops.py:1174 cond
        return cond_v2.cond_v2(pred, true_fn, false_fn, name)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/ops/cond_v2.py:91 cond_v2
        op_return_value=pred)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/framework/func_graph.py:915 func_graph_from_py_func
        func_outputs = python_func(*func_args, **func_kwargs)
    /tens

## AutoGraph and loops
AutoGraph has a few simple rules for converting loops.
- `for`: Convert if the iterable is a tensor
- `while`: Convert if the while condition depends on a tensor
If a loop is converted, it will be bynamically unrolled with `tf.while_loop`, or in the special cast of a `for x in tf.data.Dataset`, transformed into `tf.data.Dataset.reduce`.

If a loop is not converted, it will be statically unrolled

In [0]:
def test_dynamically_unrolled(f, *args):
  g = f.get_concrete_function(*args).graph
  if any(node.name == 'while' for node in g.as_graph_def().node):
    print('{}({}) uses tf.while_loop'.format(
        f.__name__, ', '.join(map(str, args))))
  elif any(node.name == 'ReduceDataset' for node in g.as_graph_def().node):
    print('{}({}) uses tf.data.Dataset.reduce'.format(
        f.__name__, ', '.join(map(str, args))))
  else:
    print('{}({}) gets unrolled.'.format(
        f.__name__, ', '.join(map(str, args))))

In [57]:
@tf.function
def for_in_range():
  x = 0
  for i in range(5):
    x += i
  return x

test_dynamically_unrolled(for_in_range)

for_in_range() gets unrolled.


In [58]:
@tf.function
def for_in_tfrange():
  x = tf.constant(0, dtype=tf.int32)
  for i in tf.range(5):
    x += i
  return x

test_dynamically_unrolled(for_in_tfrange)

for_in_tfrange() uses tf.while_loop


In [60]:
@tf.function
def for_in_tfdataset():
  x = tf.constant(0, dtype=tf.int64)
  for i in tf.data.Dataset.range(5):
    x += i
  return x

test_dynamically_unrolled(for_in_tfdataset)

for_in_tfdataset() uses tf.data.Dataset.reduce


In [61]:
@tf.function
def while_py_cond():
  x = 5
  while x > 0:
    x -= 1
  return x

test_dynamically_unrolled(while_py_cond)

while_py_cond() gets unrolled.


In [62]:
@tf.function
def while_tf_cond():
  x = tf.constant(5)
  while x > 0:
    x -= 1
  return x


test_dynamically_unrolled(while_tf_cond)

while_tf_cond() uses tf.while_loop


If you have a `break` or early `return` clause that depends on a tensor, the top-level condition or iterable should also be a tensor.


In [64]:
# Compare the following examples:
@tf.function
def while_py_true_py_break(x):
  while True: # py true
    if x == 0:
      break
    x -= 1
  return x

test_dynamically_unrolled(while_py_true_py_break, 5)

while_py_true_py_break(5) gets unrolled.


In [65]:
@tf.function
def buggy_while_py_true_tf_break(x):
  while True: # py True
    if tf.equal(x, 0): # tf.break
      break
    x -= 1
  return x

with assert_raises(TypeError):
  test_dynamically_unrolled(buggy_while_py_true_tf_break, 5)

Caught expected exception 
 <class 'TypeError'>: in converted code:

    <ipython-input-65-df1cc42ceb39>:3 buggy_while_py_true_tf_break  *
        while True: # py True
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:730 while_stmt
        return _py_while_stmt(test, body, get_state, set_state, init_vars, opts)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:845 _py_while_stmt
        while test(*loop_vars):
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/framework/ops.py:765 __bool__
        self._disallow_bool_casting()
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/framework/ops.py:531 _disallow_bool_casting
        "using a `tf.Tensor` as a Python `bool`")
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/framework/ops.py:518 _disallow_when_autograph_enabled
        " decorating it directly with @tf.function.".format(task))

    OperatorNotAllowedInGraphError: using a `tf.Tenso

In [66]:
@tf.function
def while_tf_true_tf_break(x):
  while tf.constant(True): # tf true
    if x == 0: # py break
      break
    x -= 1
  return x

test_dynamically_unrolled(while_tf_true_tf_break, 5)

while_tf_true_tf_break(5) uses tf.while_loop


In [67]:
@tf.function
def buggy_py_for_tf_break():
  x = 0
  for i in range(5):  # py for
    if tf.equal(i, 3): # tf break
      break
    x += i
  return x

with assert_raises(TypeError):
  test_dynamically_unrolled(buggy_py_for_tf_break)

Caught expected exception 
 <class 'TypeError'>: in converted code:

    <ipython-input-67-82742b0a14d0>:4 buggy_py_for_tf_break  *
        for i in range(5):  # py for
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:339 for_stmt
        return _py_for_stmt(iter_, extra_test, body, get_state, set_state, init_vars)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:348 _py_for_stmt
        if extra_test is not None and not extra_test(*state):
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/framework/ops.py:765 __bool__
        self._disallow_bool_casting()
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/framework/ops.py:531 _disallow_bool_casting
        "using a `tf.Tensor` as a Python `bool`")
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/framework/ops.py:518 _disallow_when_autograph_enabled
        " decorating it directly with @tf.function.".format(task))

    OperatorNotAllowed

In [68]:
@tf.function
def tf_for_py_break():
  x = 0
  for i in tf.range(5): # tf for
    if i == 3:  # py break
      break
    x += i
  return x

test_dynamically_unrolled(tf_for_py_break)

tf_for_py_break() uses tf.while_loop


In order to accumulate results from a dynamically unrolled loop, you'll want to use `tf.TensorArray`

In [71]:
batch_size = 2
seq_len = 3
feature_size = 4

def rnn_step(inp, state):
  return inp + state

@tf.function
def dynamic_rnn(rnn_step, input_data, initial_state):
  # [batch, time, features] -> [time, batch, features]
  input_data = tf.transpose(input_data, [1, 0, 2])
  max_seq_len = input_data.shape[0]

  states = tf.TensorArray(tf.float32, size=max_seq_len)
  state = initial_state
  for i in tf.range(max_seq_len):
    state = rnn_step(input_data[i], state)
    states = states.write(i, state)
  return tf.transpose(states.stack(), [1, 0, 2])

dynamic_rnn(rnn_step,
            tf.random.uniform([batch_size, seq_len, feature_size]),
            tf.zeros([batch_size, feature_size]))

<tf.Tensor: id=1447, shape=(2, 3, 4), dtype=float32, numpy=
array([[[0.5657548 , 0.949895  , 0.8659004 , 0.08148456],
        [1.4895302 , 1.0488229 , 1.7181417 , 0.5329131 ],
        [2.223061  , 1.9746397 , 2.582988  , 1.2302521 ]],

       [[0.37612522, 0.7070185 , 0.88283277, 0.93872   ],
        [0.5033244 , 1.6583909 , 1.1910317 , 1.0369061 ],
        [1.0986111 , 2.3352091 , 1.2790189 , 1.2901516 ]]], dtype=float32)>

As with tf.cond, tf.while_loop also comes with a number of subtleties. - Since a loop can execute 0 times, all tensors used downstream of the while_loop must be initialized above the loop - The shape/dtypes of all loop variables must stay consistent with each iteration

In [72]:
@tf.function
def buggy_loop_var_uninitialized():
  for i in tf.range(3):
    x = i
  return x

with assert_raises(ValueError):
  buggy_loop_var_uninitialized()

Caught expected exception 
 <class 'ValueError'>: in converted code:

    <ipython-input-72-815fd6bba8cc>:3 buggy_loop_var_uninitialized  *
        for i in tf.range(3):
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:315 for_stmt
        composite_symbol_names)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:419 _tf_range_for_stmt
        _disallow_undefs_into_loop(*init_vars)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:97 _disallow_undefs_into_loop
        ' before the loop: {}'.format(tuple(s.symbol_name for s in undefined)))

    ValueError: TensorFlow requires that the following symbols must be defined before the loop: ('x',)



In [73]:
@tf.function
def f():
  x = tf.constant(0)
  for i in tf.range(3):
    x = i
  return x

f()

<tf.Tensor: id=1502, shape=(), dtype=int32, numpy=2>

In [75]:
@tf.function
def buggy_loop_type_changes():
  x = tf.constant(0, dtype=tf.float32)
  for i in tf.range(3): # Yields tensors of type tf.int32...
    x = i
  return x

with assert_raises(tf.errors.InvalidArgumentError):
  buggy_loop_type_changes()

Got unexpected exception 
 <class 'TypeError'>: in converted code:

    <ipython-input-75-46359fb065eb>:4 buggy_loop_type_changes  *
        for i in tf.range(3): # Yields tensors of type tf.int32...
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:315 for_stmt
        composite_symbol_names)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:478 _tf_range_for_stmt
        opts=opts,
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:769 _tf_while_stmt
        aug_init_vars, **opts)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/ops/control_flow_ops.py:2675 while_loop
        back_prop=back_prop)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/ops/while_v2.py:198 while_loop
        add_control_dependencies=add_control_dependencies)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/framework/func_graph.py:915 func_graph_from_py_func
        func_o

In [76]:
@tf.function
def buggy_concat():
  x = tf.ones([0, 10])
  for i in tf.range(5):
    x = tf.concat([x, tf.ones([1, 10])], axis=0)
  return x

with assert_raises(ValueError):
  buggy_concat()

Caught expected exception 
 <class 'ValueError'>: in converted code:

    <ipython-input-76-bae298a1ce41>:4 buggy_concat  *
        for i in tf.range(5):
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:315 for_stmt
        composite_symbol_names)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:478 _tf_range_for_stmt
        opts=opts,
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/autograph/operators/control_flow.py:769 _tf_while_stmt
        aug_init_vars, **opts)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/ops/control_flow_ops.py:2675 while_loop
        back_prop=back_prop)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/ops/while_v2.py:198 while_loop
        add_control_dependencies=add_control_dependencies)
    /tensorflow-2.0.0/python3.6/tensorflow_core/python/framework/func_graph.py:915 func_graph_from_py_func
        func_outputs = python_func(*func_args, **func_kwargs

In [77]:
x = tf.ones([0, 10])

print(x)

tf.Tensor([], shape=(0, 10), dtype=float32)


In [78]:
@tf.function
def concat_with_padding():
  x = tf.zeros([5, 10])
  for i in tf.range(5):
    x = tf.concat([x[:i], tf.ones([1, 10]), tf.zeros([4-i, 10])], axis=0)
    x.set_shape([5, 10])
  return x
  
concat_with_padding()

<tf.Tensor: id=1628, shape=(5, 10), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>